## Data collection and imputation

In [153]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

football = Fantasy()

This is all cleaning the data

In [155]:
df = pd.read_excel('2022-offense.xlsx', sheet_name='Offense_Prior_Actuals', header=3)

# Drop columns that we don't need
drop_columns = ['GS', 'Bye', 'Notes', 'Rank', 'Y! Roto', 'Δ', 'Y! Old', 'Std', 'Δ.1', 'Std Old', 'PPR', 'Δ.2', 'PPR Old', '% Own', 'Fan Pts', 'PPG']
df = df.drop(drop_columns, axis=1)

# Change names of duplicate columns
updated_columns = {'Yds': 'Passing_Yds', 'Yds.1': 'Rushing_Yds', 'Yds.2': 'Receiving_Yds', 'Yds.3': 'Return_Yds', 
                   'TD': 'Passing_Td', 'TD.1': 'Rushing_Td', 'TD.2': 'Receiving_Td', 'TD.3': 'Return_Td', 
                   '1st': 'Passing_1st', '1st.1': 'Rushing_1st', '1st.2': 'Receiving_1st'}
df = df.rename(columns=updated_columns)


Offense_Prior_Actuals
Offense_Proj


In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

url_pro_football_reference = 'https://www.pro-football-reference.com/years/2022/passing_advanced.htm'
table_advanced = pd.read_html(url_pro_football_reference, header=1)[0]

We are not using the player contracts yet but here is where we scrape that data

In [157]:
player_contracts = table_contracts[0]
# player_contracts

We had to remove some characters that were throwing off our ability to merge the data

In [158]:
passing_advanced = table_advanced
passing_advanced = passing_advanced.head(40)
passing_advanced['Player'] = passing_advanced['Player'].str.replace('*', '')
passing_advanced['Player'] = passing_advanced['Player'].str.replace('+', '')
# passing_advanced

Here is where we collect isolate the data for quarterbacks and merge the basic dataframe with one of the advanced statistics ones.

In [159]:
# Get y - actual values from the next year (we do not have 2023 but could get them) - we will eventually need them?
QB_data = df[df["Pos"] == "QB"]

# Get the list of all column names from headers
qb_column_headers = list(QB_data.columns.values)
# print("The Column Header :", qb_column_headers)

qb_drop_columns = ['Tgt', 'Rec', 'Receiving_Yds', 'Receiving_Td', 'Receiving_1st', 'Return_Yds', 'Return_Td', '2PT',]
QB_data = QB_data.drop(qb_drop_columns, axis=1)
# QB_data


In [160]:
qb_merged = football.addAdvanced(QB_data, passing_advanced)

This is isolating data for running backs, wide receivers, and tight ends.

In [161]:
RB_data = df[df["Pos"] == "RB"]

rb_drop_columns = [ 'Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st']
RB_data = RB_data.drop(rb_drop_columns, axis=1)
# RB_data

In [162]:
WR_data = df[df["Pos"] == "WR"]

wr_drop_columns = ['Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st']
WR_data = WR_data.drop(rb_drop_columns, axis=1)
# WR_data

In [163]:
TE_data = df[df["Pos"] == "TE"]

te_drop_columns = ['Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st', 'Return_Yds', 'Return_Td']
TE_data = TE_data.drop(te_drop_columns, axis=1)
# TE_data